# Data Manipulation with Pandas 

## Categorical Types

### Exercise

* Load NFS data again. Look at a Categorical Type. Use describe.

## Date and Time Types

### Exercise

* Create a DateTime Index from NFS household data

## Merging and Joining DataFrames

### Exercise

* Join the Household NFS data with the Diary data

## Concatenation

### Exercise

* Join all of the diary data together

## Text Data Manipulation

* Maybe skip?

### Exercise

* Grab all of the food groups that contain the word milk

## Reshaping DataFrames

### Exercise

* TBD

## Pivoting

### Exercise

* TBD